# Chapter 5 — Tool Use: Executing Code

Give an agent a `BuiltInCodeExecutor` so it can write and run Python on the fly.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Create an Agent with Code Execution

In [2]:
from google.adk.agents import Agent
from google.adk.code_executors import BuiltInCodeExecutor

code_agent = Agent(
    name="code_agent",
    model="gemini-2.0-flash",
    description="An agent that can write and execute Python code.",
    instruction="""You are a coding assistant. When the user asks for calculations, data analysis,
or any task that benefits from running code, write and execute Python code to solve it.
Always show your work and explain the results.""",
    code_executor=BuiltInCodeExecutor(),
)

print("Code execution agent ready")

Code execution agent ready


## Run the Agent

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=code_agent, app_name="code_demo", session_service=session_service)

session = await session_service.create_session(app_name="code_demo", user_id="user1")

request = types.Content(
    role="user",
    parts=[types.Part(text="Calculate the first 20 Fibonacci numbers and tell me which ones are prime.")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

In [ ]:
# Try a data analysis task
request2 = types.Content(
    role="user",
    parts=[types.Part(text="Generate a random dataset of 100 numbers and compute the mean, median, standard deviation, and create a histogram description.")]
)

async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=request2):
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

## Key Takeaways

- **`BuiltInCodeExecutor`**: Lets the agent write and execute Python in a sandboxed environment
- The agent decides when code execution is needed vs. a text response
- Great for math, data analysis, and computational tasks